In [1]:
import pandas as pd
import geopandas as gpd
import time
import datetime as dt


Scooters_2023_05_17 = pd.read_csv('Data/Scooters 2023-05-17.csv')


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [2]:
Scooters_2023_05_17.head()

,Unnamed: 0,title,id,networkName,networkId,color,textColor,latitude,longitude,type,subtitle,stationProperties,time
0,0,Link Scooter,8c088413-0b87-49a6-8ece-b91b408545c1,LINK,1319,E5FF01,07000E,34.06379,-118.44610,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399
1,1,Link Scooter,76f5520c-94b7-46df-af20-c41adf63dcf1,LINK,1319,E5FF01,07000E,34.06743,-118.44965,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399
2,2,Link Scooter,5e260c79-05bb-4aeb-80e1-fe1c5d5e8639,LINK,1319,E5FF01,07000E,34.06710,-118.45234,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399
3,3,Link Scooter,0262bef4-168c-4db8-ab0a-cf5b20214107,LINK,1319,E5FF01,07000E,34.06964,-118.45061,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399
4,4,Gateway Plaza,bcycle_lametro_4613,Metro Bike Share,554,383838,FFFFFF,34.06975,-118.44535,station,2 bikes - 1 e-bike - 9 docks,"{'counts': {'bike': 2, 'electric-bike': 1, 'do...",2023-05-17 09:00:19.484399


Okay this is kind of an insane way to fix things, so I need to right this down for future readers. 

The problem at hand is some of the first data sets did not include a time variable, so its impossible to know when the data was taken. But our first unamed column always resets to zero after each call to the API. 

'Scooters 2023-05-17.csv' is the most complete dataset that I chose. The goal is to merge the time column onto the data that is missing that info. However since not all the calls ot the API started at 9am, but they all ended at 9:59pm.

To account for this, I made a column called 'cycle' which tells you which # call to the API the Data is from. But to account for some of the missing morning data, we take the largest number of calls possible (75) and subtract it by the number of call we are on.

We create a reverse_cycle column for each of the datasets missing its time column, and merge it with our complete dataset's column and we're done!

In [13]:
#Scooters_2023_05_17['cycle'] = Scooters_2023_05_17.groupby(['Unnamed: 0']).cumcount() + 1
Scooters_2023_05_17['reverse_cycle'] = Scooters_2023_05_17.groupby(['Unnamed: 0']).cumcount(ascending=False)
#Scooters_2023_05_11['reverse_cycle'] = Scooters_2023_05_11.groupby(['Unnamed: 0']).cumcount() - 75 
#Scooters_2023_05_17.assign(reverse_cycle=lambda x: n x.temp_c * 9 / 5 + 32)
print(Scooters_2023_05_17.max())
Scooters_2023_05_17.head()

Unnamed: 0                                        684
title                                 Wyton Dr Corral
id               ffff7fc0-19ca-4eb1-a87f-fbf43c495319
networkName                          Metro Bike Share
networkId                                        1340
color                                          EC39BA
textColor                                      FFFFFF
latitude                                    34.077785
longitude                                 -118.433907
type                                          station
time                       2023-05-17 21:59:24.977470
reverse_cycle                                      74
dtype: object


C:\Users\Matucoco\AppData\Local\Temp\ipykernel_17132\2503025971.py:5: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(Scooters_2023_05_17.max())


,Unnamed: 0,title,id,networkName,networkId,color,textColor,latitude,longitude,type,subtitle,stationProperties,time,reverse_cycle
0,0,Link Scooter,8c088413-0b87-49a6-8ece-b91b408545c1,LINK,1319,E5FF01,07000E,34.06379,-118.44610,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399,74
1,1,Link Scooter,76f5520c-94b7-46df-af20-c41adf63dcf1,LINK,1319,E5FF01,07000E,34.06743,-118.44965,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399,74
2,2,Link Scooter,5e260c79-05bb-4aeb-80e1-fe1c5d5e8639,LINK,1319,E5FF01,07000E,34.06710,-118.45234,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399,74
3,3,Link Scooter,0262bef4-168c-4db8-ab0a-cf5b20214107,LINK,1319,E5FF01,07000E,34.06964,-118.45061,electric-scooter,NaN,NaN,2023-05-17 09:00:19.484399,74
4,4,Gateway Plaza,bcycle_lametro_4613,Metro Bike Share,554,383838,FFFFFF,34.06975,-118.44535,station,2 bikes - 1 e-bike - 9 docks,"{'counts': {'bike': 2, 'electric-bike': 1, 'do...",2023-05-17 09:00:19.484399,74


In [19]:
Time_Cycle = pd.DataFrame()
Time_Cycle['reverse_cycle'] = Scooters_2023_05_17['reverse_cycle']
Time_Cycle['time'] = Scooters_2023_05_17['time']
Time_Cycle = Time_Cycle.drop_duplicates(subset=['time'])
print(Time_Cycle)
Time_Cycle.tail(10)

       reverse_cycle                        time
0                 74  2023-05-17 09:00:19.484399
493               73  2023-05-17 09:10:51.036629
1014              72  2023-05-17 09:21:22.380851
1549              71  2023-05-17 09:31:53.749638
2091              70  2023-05-17 09:42:25.160216
2640              69  2023-05-17 09:52:56.584437
3210              68  2023-05-17 10:03:27.995232
3800              67  2023-05-17 10:13:59.511596
4407              66  2023-05-17 10:24:31.020067
5022              65  2023-05-17 10:35:02.395071
5643              64  2023-05-17 10:45:33.785682
6268              63  2023-05-17 10:56:05.255035
6890              62  2023-05-17 11:06:36.682460
7536              61  2023-05-17 11:17:08.150651
8188              60  2023-05-17 11:27:39.654097
8845              59  2023-05-17 11:38:11.139346
9497              58  2023-05-17 11:48:42.634296
10146             57  2023-05-17 11:59:14.106646
10795             56  2023-05-17 12:09:45.598326
11450             55

,reverse_cycle,time
41430,9,2023-05-17 20:24:36.878425
42017,8,2023-05-17 20:35:09.255004
42601,7,2023-05-17 20:45:41.521801
43185,6,2023-05-17 20:56:13.278215
43789,5,2023-05-17 21:06:45.377154
44392,4,2023-05-17 21:17:17.157116
44990,3,2023-05-17 21:27:49.080371
45563,2,2023-05-17 21:38:20.923368
46118,1,2023-05-17 21:48:53.187491
46674,0,2023-05-17 21:59:24.977470


In [5]:
Time_Cycle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 16151
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   reverse_cycle  75 non-null     int64 
 1   time           75 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


In [6]:
Scooters_2023_05_11 = pd.read_csv('Data/Scooters 2023-05-11.csv')
Scooters_2023_05_11['reverse_cycle'] = Scooters_2023_05_11.groupby(['Unnamed: 0']).cumcount() - 75 

Scooters_2023_05_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50393 entries, 0 to 50392
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.1       50393 non-null  int64  
 1   Unnamed: 0         50393 non-null  int64  
 2   title              50393 non-null  object 
 3   id                 50393 non-null  object 
 4   networkName        50393 non-null  object 
 5   networkId          50393 non-null  int64  
 6   color              50393 non-null  object 
 7   textColor          50393 non-null  object 
 8   latitude           50393 non-null  float64
 9   longitude          50393 non-null  float64
 10  type               50393 non-null  object 
 11  subtitle           221 non-null    object 
 12  stationProperties  221 non-null    object 
 13  reverse_cycle      50393 non-null  int64  
 14  time               50393 non-null  object 
dtypes: float64(2), int64(4), object(9)
memory usage: 5.8+ MB


In [7]:
#testing cell, please ignore

#Scooters_2023_05_11 = pd.read_csv('Data/Scooters 2023-05-11.csv')
#Scooters_2023_05_11['reverse_cycle'] = Scooters_2023_05_11.groupby(['Unnamed: 0']).cumcount() - 75 



Scooters_2023_05_11_merge = Scooters_2023_05_11.merge(Time_Cycle, how = 'left', on = 'reverse_cycle')
Scooters_2023_05_11_merge.info()
Scooters_2023_05_11_merge.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50393 entries, 0 to 50392
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.1       50393 non-null  int64  
 1   Unnamed: 0         50393 non-null  int64  
 2   title              50393 non-null  object 
 3   id                 50393 non-null  object 
 4   networkName        50393 non-null  object 
 5   networkId          50393 non-null  int64  
 6   color              50393 non-null  object 
 7   textColor          50393 non-null  object 
 8   latitude           50393 non-null  float64
 9   longitude          50393 non-null  float64
 10  type               50393 non-null  object 
 11  subtitle           221 non-null    object 
 12  stationProperties  221 non-null    object 
 13  reverse_cycle      50393 non-null  int64  
 14  time_x             50393 non-null  object 
 15  time_y             0 non-null      object 
dtypes: float64(2), int64(4

,Unnamed: 0.1,Unnamed: 0,title,id,networkName,networkId,color,textColor,latitude,longitude,type,subtitle,stationProperties,reverse_cycle,time_x,time_y
0,0,0,Link Scooter,decba7fc-debf-472a-8d51-2b7244d063e6,LINK,1319,E5FF01,07000E,34.06588,-118.44820,electric-scooter,NaN,NaN,-75,2023-05-17 09:00:19.484399,NaN
1,1,1,Link Scooter,808f4162-124f-48cc-a026-f17f61c2dcad,LINK,1319,E5FF01,07000E,34.06377,-118.44517,electric-scooter,NaN,NaN,-75,2023-05-17 09:00:19.484399,NaN
2,2,2,Link Scooter,688ad54a-72e5-43d2-ad6f-690fac0c02ae,LINK,1319,E5FF01,07000E,34.06960,-118.43841,electric-scooter,NaN,NaN,-75,2023-05-17 09:00:19.484399,NaN
3,3,3,Link Scooter,bb467505-a28f-4f80-ae8c-0b1fc3b6e584,LINK,1319,E5FF01,07000E,34.06377,-118.44513,electric-scooter,NaN,NaN,-75,2023-05-17 09:00:19.484399,NaN
4,4,4,Link Scooter,b7cc8424-a9d2-4fe7-ba70-859639d39584,LINK,1319,E5FF01,07000E,34.06563,-118.44002,electric-scooter,NaN,NaN,-75,2023-05-17 09:00:19.484399,NaN


In [20]:
#Ignore my ugly code, I know I should do a for loop but I'm lazy
Scooters_2023_05_10 = pd.read_csv('Data/Scooters 2023-05-10.csv')
Scooters_2023_05_11 = pd.read_csv('Data/Scooters 2023-05-11.csv')
Scooters_2023_05_12 = pd.read_csv('Data/Scooters 2023-05-12.csv')
Scooters_2023_05_15 = pd.read_csv('Data/Scooters 2023-05-15.csv')
Scooters_2023_05_16 = pd.read_csv('Data/Scooters 2023-05-16.csv')

Scooters_2023_05_10 = Scooters_2023_05_10.drop(columns=['reverse_cycle', 'time'])
Scooters_2023_05_11 = Scooters_2023_05_11.drop(columns=['reverse_cycle', 'time'])
Scooters_2023_05_12 = Scooters_2023_05_12.drop(columns=['reverse_cycle', 'time'])
Scooters_2023_05_15 = Scooters_2023_05_15.drop(columns=['reverse_cycle', 'time'])
Scooters_2023_05_16 = Scooters_2023_05_16.drop(columns=['reverse_cycle', 'time'])

Scooters_2023_05_10['reverse_cycle'] = Scooters_2023_05_10.groupby(['Unnamed: 0']).cumcount(ascending=False) 
Scooters_2023_05_11['reverse_cycle'] = Scooters_2023_05_11.groupby(['Unnamed: 0']).cumcount(ascending=False) 
Scooters_2023_05_12['reverse_cycle'] = Scooters_2023_05_12.groupby(['Unnamed: 0']).cumcount(ascending=False) 
Scooters_2023_05_15['reverse_cycle'] = Scooters_2023_05_15.groupby(['Unnamed: 0']).cumcount(ascending=False) 
Scooters_2023_05_16['reverse_cycle'] = Scooters_2023_05_16.groupby(['Unnamed: 0']).cumcount(ascending=False) 

Scooters_2023_05_10 = Scooters_2023_05_10.merge(Time_Cycle,how = 'right', on = 'reverse_cycle')
Scooters_2023_05_11 = Scooters_2023_05_11.merge(Time_Cycle,how = 'right', on = 'reverse_cycle')
Scooters_2023_05_12 = Scooters_2023_05_12.merge(Time_Cycle,how = 'right', on = 'reverse_cycle')
Scooters_2023_05_15 = Scooters_2023_05_15.merge(Time_Cycle,how = 'right', on = 'reverse_cycle')
Scooters_2023_05_16 = Scooters_2023_05_16.merge(Time_Cycle,how = 'right', on = 'reverse_cycle')

Scooters_2023_05_10.to_csv('Scooters 2023-05-10.csv')
Scooters_2023_05_11.to_csv('Scooters 2023-05-11.csv')
Scooters_2023_05_12.to_csv('Scooters 2023-05-12.csv')
Scooters_2023_05_15.to_csv('Scooters 2023-05-15.csv')
Scooters_2023_05_16.to_csv('Scooters 2023-05-16.csv')